# Settings

In [3]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [4]:
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.ensemble import RandomForestClassifier

In [6]:
os.chdir('/content/gdrive/My Drive/피셋 방세/dataset') # Set your directory

In [7]:
def cost_scorer (actual, pred):
    """
    True pos - predicted pos : Zero,
    True neg - predicted neg : Zero,
    True pos - predicted neg : 500,
    True neg - predicted pos : 10
    """
    
    cost=[]

    for i in range(0,len(actual)):
        if actual[i] == pred[i]:
                cost.append(0)
        else:
            if pred[i]=='neg':
                cost.append(500)
            else:
                cost.append(10)

    return sum(cost)

실제 값과 pred값을 예측해줄 함수

In [8]:
def custom_cost (y_pred, dataset):
    y_true = dataset.get_label()
    
    cost = []
    for i in range(0,len(y_true)):
        if y_true[i] == y_pred[i]:
            cost.append(0)
        else:
            if y_pred[i] == 0:
                cost.append(500)
            else:
                cost.append(10)
    return 'cost', -sum(cost), True

LGBM 시 cost 계산 해 줄 함수

# 사용할 Data load

knn impute + smote

In [20]:
# knn imputation과 SMOTE한 x값에 대응될 y값
train_y = pd.read_csv('knn_y_smote_final2.csv') #
train_y.drop('Unnamed: 0', axis=1, inplace=True)

# knn imputation과 SMOTE한 X값에 해당하는 데이터 (train_feature)
train = pd.read_csv('knn_nopca_x_smote_final2.csv') #
train.drop('Unnamed: 0', axis=1, inplace=True)

In [21]:
train

,aa_000,ac_000,ad_000,ae_000,af_000,ag_000,ag_001,ag_002,ag_003,ag_004,ag_005,ag_006,ag_007,ag_008,ag_009,ah_000,ai_000,aj_000,ak_000,al_000,am_0,an_000,ao_000,ap_000,aq_000,ar_000,as_000,at_000,au_000,av_000,ax_000,ay_000,ay_001,ay_002,ay_003,ay_004,ay_005,ay_006,ay_007,ay_008,...,db_000,dc_000,dd_000,de_000,df_000,dg_000,dh_000,di_000,dj_000,dk_000,dl_000,dm_000,dn_000,do_000,dp_000,dq_000,dr_000,ds_000,dt_000,du_000,dv_000,dx_000,dy_000,dz_000,ea_000,eb_000,ec_00,ed_000,ee_000,ee_001,ee_002,ee_003,ee_004,ee_005,ee_006,ee_007,ee_008,ee_009,ef_000,eg_000
0,0.048059,2.000001e-01,3.518051e-08,0.0,0.0,0.000000,0.000000,0.000225,0.011373,0.041520,0.038264,0.027355,0.013495,0.003926,0.000055,0.055147,0.000040,0.000096,0.000000,0.005987,0.006465,0.057755,0.058066,0.021216,0.028155,0.004571,0.0,0.000000,0.0,0.000707,0.016120,0.000000,0.000000,0.000000,0.000000,0.000001,0.000075,0.001255,0.048154,0.023928,...,0.001764,0.043998,0.008509,0.004918,0.000000,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.0,0.020170,0.026999,0.054671,0.001557,0.023995,0.027756,0.036754,0.062564,0.034593,0.018175,0.005264,0.0,0.0,0.037996,0.020830,0.030157,0.021147,0.041542,0.030780,0.013466,0.014419,0.008260,0.045375,0.018581,0.001518,9.480121e-05,0.0,0.0
1,0.013858,4.000000e-01,3.508732e-08,0.0,0.0,0.000000,0.000000,0.000000,0.000063,0.000875,0.011007,0.014495,0.006020,0.001121,0.000028,0.020799,0.000016,0.000000,0.000000,0.000016,0.000021,0.021057,0.020524,0.007947,0.010144,0.000000,0.0,0.000000,0.0,0.000725,0.001108,0.000000,0.000000,0.000000,0.000000,0.000000,0.000498,0.007849,0.007796,0.005187,...,0.002268,0.019121,0.004369,0.001113,0.000000,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.0,0.006057,0.016867,0.027140,0.000000,0.000000,0.014182,0.019928,0.007860,0.004605,0.004200,0.000025,0.0,0.0,0.002692,0.010361,0.015134,0.006045,0.005944,0.013559,0.010304,0.015221,0.007483,0.004448,0.002699,0.006544,1.071684e-03,0.0,0.0
2,0.025172,3.981778e-07,7.940079e-08,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000551,0.011123,0.029994,0.011815,0.000814,0.000012,0.035532,0.000000,0.000000,0.000000,0.000005,0.000009,0.035340,0.034309,0.009554,0.017745,0.000000,0.0,0.000000,0.0,0.001071,0.013346,0.000000,0.000000,0.000000,0.000000,0.000000,0.000361,0.014261,0.011522,0.010271,...,0.005041,0.025414,0.006029,0.002202,0.000000,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.0,0.010709,0.031804,0.035485,0.000000,0.000000,0.029729,0.045659,0.002849,0.004775,0.000000,0.000000,0.0,0.0,0.009309,0.014481,0.017559,0.008524,0.008759,0.025004,0.018785,0.025862,0.009181,0.014058,0.008079,0.010812,8.181460e-04,0.0,0.0
3,0.146131,1.283893e-06,1.190080e-07,0.0,0.0,0.000000,0.000000,0.000825,0.031306,0.120125,0.162819,0.114758,0.063676,0.041783,0.003198,0.228817,0.018777,0.000023,0.000000,0.015210,0.017040,0.236473,0.221848,0.138027,0.164037,0.011429,0.0,0.000312,0.0,0.007683,0.029496,0.000000,0.000000,0.000000,0.000000,0.000052,0.000333,0.020868,0.188662,0.075920,...,0.000000,0.164346,0.098128,0.018034,0.000000,0.000000,2.508082e-04,0.011344,0.0,0.0,0.0,0.0,0.129452,0.079769,0.107423,0.004210,0.063573,0.217264,0.233926,0.104360,0.158640,0.056738,0.018055,0.0,0.0,0.090274,0.078121,0.108181,0.139086,0.108351,0.138298,0.090661,0.092342,0.032152,0.094218,0.109121,0.004311,2.981566e-05,0.0,0.0
4,0.012351,2.000001e-01,2.520882e-08,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000331,0.007860,0.013894,0.007299,0.001976,0.000162,0.018546,0.005440,0.000012,0.000000,0.000002,0.000002,0.017522,0.015244,0.007929,0.015359,0.000000,0.0,0.000178,0.0,0.001101,0.001420,0.000000,0.000000,0.000000,0.000000,0.000000,0.000819,0.008901,0.006110,0.003537,...,0.005293,0.013530,0.004299,0.001190,0.000000,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.0,0.008127,0.019688,0.023092,0.000000,0.000000,0.011702,0.014666,0.008493,0.005601,0.000000,0.000000,0.0,0.0,0.007069,0.010161,0.012347,0.005534,0.005659,0.013884,0.011702,0.016230,0.005303,0.006124,0.001175,0.002617,5.061314e-04,0.0,0.0
...,...,...,...,...,...,...,

In [22]:
train_y

,class
0,neg
1,neg
2,neg
3,neg
4,neg
...,...
78135,pos
78136,pos
78137,pos
78138,pos


#  Modeling
LGBM: train set만을 이용하여 cost 구하기

In [23]:
from lightgbm import LGBMClassifier

## 1. test / val data split

In [24]:
train_y [ 'class_int'] =[ 0  if train_y['class'][i] == 'neg' else 1
                              for i in range(0,len(train_y))
                            ]

predict를 위해서 neg는 0, pos는 1로 만든 column을 만들어줌  
(pos/neg의 str 상태로는 오류가 떠서,,)

In [25]:
X_train, X_val, y_train, y_val = train_test_split(train,
                                                        train_y['class_int'],
                                                        test_size=0.3,
                                                        shuffle=True,
                                                        random_state = 500)

# 2. 모델 학습

In [26]:
lg = LGBMClassifier(n_estimators=1000, metric = "custom_cost")  # 위에서 정의한 함수로 metric 평가
lg.fit(X_train, y_train, eval_set = (X_val, y_val))

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               metric='custom_cost', min_child_samples=20,
               min_child_weight=0.001, min_split_gain=0.0, n_estimators=1000,
               n_jobs=-1, num_leaves=31, objective=None, random_state=None,
               reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)

# 3. validationset을 이용한 Predict

In [27]:
pred = lg.predict(X_val)

In [28]:
pred_df = pd.DataFrame(pred)

In [29]:
pred_df

,0
0,0
1,0
2,1
3,1
4,1
...,...
23437,1
23438,1
23439,1
23440,0


예측이 0과 1로 되어 있음 --> 0과 1로 학습했기 때문에 예측도 0과 1로 되어 있지만  
최종 제출은 pos / neg로 해야 하기 때문에 pos / neg column만듦

In [30]:
pred_df = pd.DataFrame(pred)
pred_df['class'] = [ 'pos' if pred_df[0][i]>0.5 else 'neg' for i in range(0,len(pred))]
pred_df

,0,class
0,0,neg
1,0,neg
2,1,pos
3,1,pos
4,1,pos
...,...,...
23437,1,pos
23438,1,pos
23439,1,pos
23440,0,neg


이 때 0.5 이상은 pos, 0.5 이하는 neg로 할당시켰음

# 4. LGBM Cost 구하기

In [31]:
y_val

39248    0
8917     0
73226    1
69195    1
50841    1
        ..
46836    1
52959    1
66810    1
12169    0
43453    1
Name: class_int, Length: 23442, dtype: int64

y_val의 index가 엉망인데 cost_scorer가 인덱스 기준으로 돌아가서 index reset 해줘야함

In [32]:
y_val_tmp = y_val.reset_index(drop=True)
y_val_tmp = pd.DataFrame(y_val_tmp)
y_val_tmp

,class_int
0,0
1,0
2,1
3,1
4,1
...,...
23437,1
23438,1
23439,1
23440,0


In [33]:
y_val_tmp ['class'] = [ 'pos' if y_val_tmp['class_int'][i]>0.5 else 'neg' for i in range(0,len(y_val_tmp))]
y_val_tmp

,class_int,class
0,0,neg
1,0,neg
2,1,pos
3,1,pos
4,1,pos
...,...,...
23437,1,pos
23438,1,pos
23439,1,pos
23440,0,neg


In [34]:
cost_scorer(y_val_tmp['class'], pred_df['class'])

940

# 5. 모델저장

In [35]:
import joblib

In [36]:
joblib.dump(lg, 'final_model.pkl')

['final_model.pkl']